In [1]:
!pip install --upgrade pythainlp
!pip install pyLDAvis

In [6]:
import pandas as pd
import pythainlp
import gensim
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [7]:
df = pd.read_csv('/content/Sunscreen_review.csv')

In [8]:
df.tail()

,Review ID,Review
5,6,จัดส่งไวขนส่งบริการดีโทรมาบอกก่อนเข้ามาส่งสินค...
6,7,ลองใช่แล้วดีมาก ไม่ทำให้หน้ามัน กันแดดได้ดี ไม...
7,8,การจัดส่งรวดเร็ว​ดีมาก​มาก​ แพ็คสินค้า​มาดี​ ไ...
8,9,ดีมากค่ะชอบมาก ราคาดีส่งไวมากถูกใจสุดๆไปเลยค่ะ...
9,10,ได้รับสินค้าเรียบร้อยแล้วอย่างรวดเร็ว สั่งเมื่...


In [79]:
stopwords = list(pythainlp.corpus.thai_stopwords())
removed_words = [' ', '  ', '/n', 'ร้าน', '(', ')', '.', 'กก', 'ๆ']
screening_words = stopwords + removed_words

def tokenize_with_space(sentence):
  merged = ''
  words = pythainlp.word_tokenize(str(sentence), engine='newmm')
  for word in words:
    if word not in screening_words:
      merged = merged + ',' + word
  return merged[1:]

In [80]:
df['Review_tokenized'] = df['Review'].apply(lambda x: tokenize_with_space(x))

In [81]:
df['Review_tokenized'] = df['Review_tokenized'].str.replace('แพ,ค', 'แพ็ค')

In [82]:
df

,Review ID,Review,Review_tokenized
0,1,ราคาดีค่ะ แพคมาดี ขนส่งสินค้า​ก็ไวดี . ต้อง​ลอ...,"ราคา,ดี,แพ็ค,ดี,ขนส่งสินค้า,​,ไว,ดี,​,ลอง,​,​,..."
1,2,ร้านส่งเร็วสุดๆ ได้ของแถม3อย่าง ครีมอาบน้ำลิป ...,"ของแถม,3,ครีม,อาบน้ำ,ลิป,กระเป๋า,ถูกใจ,เนื้อ,ซ..."
2,3,ได้รับสินค้าแล้วนะคะ ส่งไวมากค่ะ รอไม่นาน ราคา...,"สินค้า,นะคะ,ไว,รอ,ราคา,ห้าง,ไม่ต้อง,ข้างนอก,สั..."
3,4,ราคาดี จัดส่งไว แพ็คสินค้ามาดี ซื้อตามรีวิว,"ราคา,ดี,จัดส่ง,ไว,แพ็ค,สินค้า,ดี,ซื้อ,รีวิว"
4,5,หีบห่อมาดีมาก ได้ของครบ และมีของแถมมาด้วย สินค...,"หีบห่อ,ดีมาก,ของแถม,สินค้า,หมดอายุ,ดีมาก,แพคเก..."
5,6,จัดส่งไวขนส่งบริการดีโทรมาบอกก่อนเข้ามาส่งสินค...,"จัดส่ง,ไว,ขนส่ง,บริการ,ดี,โทร,เข้ามา,ส่งสินค้า..."
6,7,ลองใช่แล้วดีมาก ไม่ทำให้หน้ามัน กันแดดได้ดี ไม...,"ลอง,ดีมาก,ทำ,ให้หน้า,กันแดด,ได้ดี,แพ้,หม่,สิว,..."
7,8,การจัดส่งรวดเร็ว​ดีมาก​มาก​ แพ็คสินค้า​มาดี​ ไ...,"การจัดส่ง,​,ดีมาก,​,​,แพ็ค,สินค้า,​,ดี,​,ความเ..."
8,9,ดีมากค่ะชอบมาก ราคาดีส่งไวมากถูกใจสุดๆไปเลยค่ะ...,"ดีมาก,ชอบ,ราคา,ดี,ไว,ถูกใจ,5,ดาว,เล,ยาคะ,ส่วนต..."
9,10,ได้รับสินค้าเรียบร้อยแล้วอย่างรวดเร็ว สั่งเมื่...,"สินค้า,เรียบร้อย,สั่ง,รุ่งขึ้น,เยี่ยม,สินค้า,ส..."


In [83]:
documents = df['Review_tokenized'].to_list()
texts = [[text for text in doc.split(',')] for doc in documents]
dictionary = gensim.corpora.Dictionary(texts)

In [84]:
print(dictionary.token2id.keys())

dict_keys(['ขนส่งสินค้า', 'คน', 'ดี', 'ดูก่อน', 'ตา', 'น่ารัก', 'ปก', 'ราคา', 'รีวิว', 'ลอง', 'เป็นไง', 'แพ็ค', 'ไว', '\u200b', '\u200bจก', '3', 'กระเป๋า', 'ของแถม', 'ครีม', 'ซึม', 'ซื้อ', 'ดีมาก', 'ถูกใจ', 'พลาด', 'มันวาว', 'ลังเล', 'ลิป', 'ห้าม', 'อาบน้ำ', 'เนื้อ', 'ๆๆๆๆ', 'กลับมา', 'ข้างนอก', 'นะคะ', 'รอ', 'สั่ง', 'สินค้า', 'ห้าง', 'ไม่ต้อง', 'จัดส่ง', 'ชอบ', 'ดูดี', 'ทดลอง', 'ผลิตภัณฑ์', 'ระยะยาว', 'หมดอายุ', 'หีบห่อ', 'แพคเกจ', 'ขนส่ง', 'ความเสียหาย', 'บริการ', 'ฟรี', 'ส่งสินค้า', 'เข้ามา', 'โค๊ด', 'โทร', 'กันแดด', 'ทำ', 'สิว', 'หม่', 'อุดหนุน', 'แน่นอน', 'แพ้', 'ให้หน้า', 'ได้ดี', 'การขนส่ง', 'การจัดส่ง', 'คุณภาพ', 'คุ้มค่า', 'เหมาะ', '5', 'ดาว', 'ตัว', 'ถู', 'นิ้วโป้ง', 'ยาคะ', 'ว่าแล้ว', 'สิบ', 'ส่วนตัว', 'เจอ', 'เล', 'รุ่งขึ้น', 'เยี่ยม', 'เรียบร้อย'])


In [85]:
gensim_corpus = [dictionary.doc2bow(text, allow_update=True) for text in texts]
word_frequencies = [[(dictionary[id], frequence) for id, frequence in couple] for couple in gensim_corpus]

Topic Modeling

In [89]:
num_topics = 10
chunksize = 300
passes = 20
iterations = 25
eval_every = 1

temp = dictionary[0]
id2word = dictionary.id2token
model= gensim.models.LdaModel(corpus = gensim_corpus, id2word = id2word, chunksize = chunksize, \
                                    alpha = 'auto', eta = 'auto', \
                                    iterations = iterations, num_topics = num_topics, \
                                    passes = passes, eval_every = eval_every)

In [90]:
vis_data = pyLDAvis.gensim.prepare(model, gensim_corpus, dictionary, mds = 'mmds')

/usr/local/lib/python3.10/dist-packages/sklearn/manifold/_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


In [91]:
pyLDAvis.gensim.prepare(model, gensim_corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.241082  0.012586       1        1  31.240926
8      0.080613  0.172606       2        1  24.049701
0     -0.098248  0.015307       3        1  14.902625
2      0.090716 -0.066824       4        1   8.492880
3      0.044846  0.033205       5        1   8.488344
4      0.016861 -0.071071       6        1   7.175001
9      0.035934 -0.108795       7        1   5.215956
1      0.023490  0.004297       8        1   0.150022
5      0.023375  0.004358       9        1   0.145592
7      0.023495  0.004330      10        1   0.138954, topic_info=      Term       Freq      Total Category  logprob  loglift
13       ​  16.000000  16.000000  Default  30.0000  30.0000
36  สินค้า   5.000000   5.000000  Default  29.0000  29.0000
2       ดี   9.000000   9.000000  Default  28.0000  28.0000
21   ดีมาก   4.000000   4.000000  Default  27.0000  27.0000
22   ถูกใจ   2.000000   2.000000  Default  26.0000  26.0000
..     ...        ...        ...      ...      ...      ...
22   ถูกใจ   0.002531   2.904564  Topic10  -4.4308  -0.4667
23    พลาด   0.002531   1.283498  Topic10  -4.4308   0.3500
24  มันวาว   0.002531   1.283509  Topic10  -4.4308   0.3500
25   ลังเล   0.002531   1.283501  Topic10  -4.4308   0.3500
26     ลิป   0.002531   1.283502  Topic10  -4.4308   0.3500

[506 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
15        2  0.779116        3
70        2  0.779099        5
16        2  0.779119  กระเป๋า
31        6  0.962518   กลับมา
56        4  0.925818   กันแดด
...     ...       ...      ...
12        3  0.482033       ไว
12        6  0.241017       ไว
30        2  0.955103     ๆๆๆๆ
13        1  0.965290        ​
14        1  0.757309      ​จก

[112 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 9, 1, 3, 4, 5, 10, 2, 6, 8])